# Preprocessing Stage
### Processing 2020 Taxi data 

__This will include the following process:__

1- Reading the Taxi data stored in AWS S3 bucket.

2- Analyzing the data and cleaning the data.

3- Grouping the data in Hourly basis and Make it ready for Modeling

4- Saving the transformed data in S3 bucket



===============================================================================================




# 1- Loading the data

In [ ]:
# starting pyspark session

In [1]:
from pyspark.sql import SparkSession

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1662022518326_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName('cleaning').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* __2020 taxi consist of 12 parquet files, each file contain a month taxi records data. The files are uploaded in `taxi-project-thesis` AWS Bucket inside `taxi2020` folder. To read all the 12 parquet files together we will use `.coalesce(1)` function in pyspark__

In [3]:
# reading 12 parquet files together
df=spark.read.parquet("s3://taxi-project-thesis/taxi2020/*").coalesce(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24649092

__The dataframe contains `24649092` taxi trips. Lets analyze these trips.__

In [7]:
# checking the features schema
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)

In [8]:
from pyspark.sql.functions import substring, to_timestamp,hour,dayofyear,month,year,weekofyear,date_format

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 2- Analyzing  and cleaning the data
## 2.1 Cleaning incorrect trips using pickup_datetime and dropoff_datetime

__We need to check are all the trips in the dataframe belong to 2020 ?__

* To do that, we will need to create new features from ` Pick-up year`,`Drop-off year`


In [9]:
# adding the pick-up year
df_new=df.withColumn('Pick_Year',year(df['tpep_pickup_datetime']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# adding the drop-off year
df_new=df_new.withColumn('Drop_Year',year(df_new['tpep_dropoff_datetime']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Cleaning the trips using the pick-up year
* We need to extract the unique pick year to see if there are trips available in the dataframe which belong to incorrect years

In [11]:
# collecting the distinct values availble in the pick-up year
df_new.select('Pick_Year').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Pick_Year=2020), Row(Pick_Year=2009), Row(Pick_Year=2008), Row(Pick_Year=2019), Row(Pick_Year=2021), Row(Pick_Year=2003), Row(Pick_Year=2002)]

In [14]:
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Saving these distinct values in list
pick_year = df_new.select(collect_set('Pick_Year').alias('Pick_Year')).first()['Pick_Year']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
pick_year

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2019, 2008, 2020, 2002, 2009, 2021, 2003]

__We have trips which occurred in diffrent years which is not correct. lets analyze these trips__

lets see how many trips occured in each year found in the Pick-up column

In [17]:
for i in (pick_year):
    print('year ',i,df_new.filter(df_new['Pick_Year']==i).count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

year  2019 131
year  2008 32
year  2020 24648812
year  2002 1
year  2009 90
year  2021 24
year  2003 2

__We will remove all the trips not occured in 2020. Its worth noting that there are some trips started in 2019 and ended in 2020. to understand this logic, there might be some trips picked in the last hours of 31-Dec-2019 and droped in first hour of january 2020. So we will keep 2019 trips and 2020 trips for the mean time and delete all the rest__

In [18]:
# we will filter out the year 2020 and 2019
year_list=[2019,2020]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#filtering only 2019,2020 trips data
df_new=df_new.filter(df_new.Pick_Year.isin(year_list))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df.count()-df_new.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

149

__There are 149 trips which have been deleted__

In [20]:
# now lets confirm the pick up year in our dataframe
df_new.select('Pick_Year').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Pick_Year=2020), Row(Pick_Year=2019)]

### Cleaning the trips using the drop-off year

The assumption that all the trips should be dropped in 2020 and 2021 is similar to the one mentioned above. Since there are some trips that might be picked during the last hour of the year, they should be dropped in the first hour of 2021.

* We need to extract the unique drop year to see if there are incorrect years

In [22]:
# checking the distinct values in the drop year column
df_new.select('Drop_Year').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Drop_Year=2020), Row(Drop_Year=2019), Row(Drop_Year=2021)]

__So we have observations which been dropped in 2019. This is incorrect__ only trips which been dropped in 2020 and 2021 should be kept. 

In [23]:
# we will remove the trips which been dropped in 2019
df_new.filter(df_new['Drop_Year']==2019).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

70

__Now lets check the trips dropped in 2021, is our assumption is correct?__
* To do that we will create pick-up month column from `tpep_pickup_datetime` and create Drop-off month column from `tpep_dropoff_datetime`

In [25]:
# checking how many trips have been dropped in 2021
df_new.filter((df_new['Pick_Year']==2020) & (df_new['Drop_Year']==2021)).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

181

So there are trips happened in Dec 2020 and dropped in start of Jan 2021

In [26]:
# adding pick up month
df_new=df_new.withColumn('Pick_month',month(df_new['tpep_pickup_datetime']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# adding drop month
df_new=df_new.withColumn('drop_month',month(df_new['tpep_dropoff_datetime']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_new.filter((df_new['Pick_Year']==2020) & (df_new['Drop_Year']==2021)).select(['Pick_month','drop_month']).show(n=20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+
|Pick_month|drop_month|
+----------+----------+
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
+----------+----------+
only showing top 20 rows

__SO our assumption is correct, we will keep these trips.__

Now we will remove the incorrect trips which dropped in 2019

In [30]:
year_list=[2020,2021]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_new=df_new.filter(df_new.Drop_Year.isin(year_list))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
#check the drop year after filtering
df_new.select('Drop_Year').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Drop_Year=2020), Row(Drop_Year=2021)]

In [33]:
df_new.filter(df_new['Pick_Year']==2019).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

63

63 trips have been removed. these trips are the trips dropped in 2019

__Now lets check the trips picked in 2019 and dropped in 2020__

In [34]:
df_new.filter((df_new['Pick_Year']==2019) & (df_new['Drop_Year']==2020)).select(['Pick_month','drop_month']).show(n=100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+
|Pick_month|drop_month|
+----------+----------+
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|         1|
|        12|    

In [35]:
# to make sure these 63 observations are trips picked in dec 2019 and dropped in jun 2020
df_new.filter((df_new['Pick_Year']==2019) & (df_new['Drop_Year']==2020)).select(['Pick_month','drop_month']).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

63

## 2.2 Analyzing and cleaning the trip distance


Checking the trips that have zore trip distance

In [38]:
df_new.filter(df_new['trip_distance']==0).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

330082

__We have `330082` trips with zero trip distance which is 0.13 % of our data__

lets analyze these trips further. we will have a look on `passenger_count` ,`trip_distance`,`fare_amount` to see if its correct or not

In [40]:
df_new.filter(df_new['trip_distance']==0).select(['tpep_pickup_datetime',
                                                  'tpep_dropoff_datetime','passenger_count','trip_distance',
                                                  'fare_amount']).show(200)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+---------------+-------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|fare_amount|
+--------------------+---------------------+---------------+-------------+-----------+
| 2020-04-01 00:33:50|  2020-04-01 00:34:00|            1.0|          0.0|       10.3|
| 2020-04-01 00:48:16|  2020-04-01 00:48:19|            1.0|          0.0|       12.8|
| 2020-04-01 00:48:51|  2020-04-01 00:49:15|            1.0|          0.0|       20.2|
| 2020-04-01 00:45:37|  2020-04-01 00:47:05|            1.0|          0.0|        3.0|
| 2020-04-01 00:48:41|  2020-04-01 00:49:06|            0.0|          0.0|        2.5|
| 2020-04-01 00:19:56|  2020-04-01 00:25:36|            1.0|          0.0|     -43.46|
| 2020-04-01 00:19:56|  2020-04-01 00:25:36|            1.0|          0.0|      43.46|
| 2020-04-01 00:07:31|  2020-04-01 00:23:51|            1.0|          0.0|       30.2|
| 2020-04-01 00:38:51|  2020-04-01 00:52:12

__These are the zero distance trips,  as we can see its paid trips and there are passengers. how to find the incorrect trips ?__


* My assumption is if the drop location ID is silimar to pick location ID, then we assume there was congustion and the taxi didnt move that much or it was very short trip. so these trips are normal. On the other hand, if the drop location is different than the pick-up location, then its impossible that these trips can have zero distance and these trips are incorrect.

Lets create dataframe that only have zero distance trips

In [41]:
zero_distance=df_new.filter(df_new['trip_distance']==0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
print('Number of trips with zero trip distance ',zero_distance.count(),' trips')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of trips with zero trip distance  330082  trips

In [43]:
# trips which have zero distance and pick location is similir to drop location
zero_distance.filter(zero_distance['PULocationID']==zero_distance['DOLocationID']).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

157730

__So we have 157730 trips in which the pick location is similir to drop location. We will  remove the trips which have zero trip distance and the pick location is different than drop location__

In [45]:
# trips which have zero distance and pick location is different than drop location
zero_distance.filter(zero_distance['PULocationID']!=zero_distance['DOLocationID']).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

172352

In [47]:
df_new.filter((df_new['trip_distance']==0) & (df_new['PULocationID']!=df_new['DOLocationID'])).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

172352

In [48]:
# our dataset should have this count
df_new.count() - 172352

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24476521

### Steps to remove these `172352` trips:

* We will create data having zero distance trips (already done above).

* We will create dataframe that dont have zero distance trips.(`no_zero_distance`)


* We will filter out the trips that have zero distance and the pick up location is different than the drop locations from the zero dataframe.

* Then we will merge the zero dataframe with the non-zero-distance dataframe

In [49]:
# 1- we will remove the trips contains zero distance
no_zero_distance=df_new.filter(df_new['trip_distance']!=0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
no_zero_distance.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24318791

In [51]:
#check there is no zero distance trips
no_zero_distance.filter('trip_distance=0').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [52]:
# we will create data having zero which we already created before
zero_distance.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

330082

In [53]:
#checking the count of the trips with zero distance and pick location is different than drop location
zero_distance.filter(zero_distance['PULocationID']!=zero_distance['DOLocationID']).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

172352

In [54]:
#3 - we will remove the trips that have zero distance and the pick up location is different than the drop locations
# here we will filter only the trips with similar pick and drop location
Zero_distance_cleared=zero_distance.filter(zero_distance['PULocationID']==zero_distance['DOLocationID'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
Zero_distance_cleared.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

157730

In [56]:
# to varify the correct number
no_zero_distance.count() +Zero_distance_cleared.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24476521

In [57]:
# we will merge the trips that have no zero distance with the trips that have zero distance and simlar pick and drop locations
df_new_1 = no_zero_distance.union(Zero_distance_cleared)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
# this dataset contain all the observation except with zero distance and drop location is different than pick location
df_new_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24476521

In [59]:
#Check if we have trips with zero distance and diffrent drop and pick locations
df_new_1.filter((df_new_1['trip_distance']==0) & (df_new_1['PULocationID']!=df_new_1['DOLocationID'])).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

__Now we cleaned the trips that have zero distance and pick-up location is different than the drop-off location__

Lets move on to check the statistical summary of `trip_distance` column

In [60]:
df_new_1.select('trip_distance').describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|     trip_distance|
+-------+------------------+
|  count|          24476521|
|   mean|3.5519427744656715|
| stddev| 326.1756239822378|
|    min|            -30.62|
|    max|         350914.89|
+-------+------------------+

We have negative trip distance which is incorrect, lets analyze the count and remove this trips.

In [61]:
df_new_1.filter(df_new_1['trip_distance']<0).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2338

In [62]:
# We will remove these trips
df_new_1=df_new_1.filter(df_new_1['trip_distance']>=0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Checking the outliers__

In [63]:
 # Q1 : First Quartile ., Q3 : Third Quartile
Q1 = df_new_1.approxQuantile('trip_distance',[0.25],relativeError=0.05)
Q3 = df_new_1.approxQuantile('trip_distance',[0.75],relativeError=0.05)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
print('Q1: ',Q1, 'Q3: ',Q3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Q1:  [1.0] Q3:  [2.8]

In [65]:
df_new_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24474183

In [66]:
df_new_1.filter(df_new_1['trip_distance']>2.8).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6672306

__We have 75% of our trips have trip distance below 2.8 mile__

In [67]:
#IQR : Inter Quantile Range
#We need to define the index [0], as Q1 & Q3 are a set of lists
IQR = Q3[0] - Q1[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
#selecting the data, with -1.5*IQR to + 1.5*IQR., where param = 1.5 default value
less_Q1 =  Q1[0] - 1.5*IQR
more_Q3 =  Q3[0] + 1.5*IQR

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
more_Q3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5.5

In [70]:
df_new_1.filter(df_new_1['trip_distance']>more_Q3).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2970967

__Statistically, any trip have more than 5.5 mile trip distance is an outlier. We have around 12% of our data contains trips above 5.5 mile. But in reality trips more than 5.5 doesnt sound like incorrect. and outliers will not affect our model since we are going to aggregate the counts of trips. so we will just remove the unrealistic values__

In [71]:
df_new_1.groupBy('trip_distance').count().orderBy(df_new_1['trip_distance'].desc()).show(50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|trip_distance|count|
+-------------+-----+
|    350914.89|    1|
|    350814.14|    1|
|     350793.6|    1|
|    350722.34|    1|
|    350696.98|    1|
|    350104.58|    1|
|    349987.05|    1|
|     349692.3|    1|
|    297004.51|    1|
|    275196.59|    1|
|    269803.73|    1|
|    256305.74|    1|
|    256069.13|    1|
|    239679.02|    1|
|    231147.49|    1|
|    222795.31|    1|
|    220386.23|    1|
|    210240.07|    1|
|    210148.62|    1|
|    180827.51|    1|
|    178867.75|    1|
|     176229.2|    1|
|    175262.86|    1|
|    174549.75|    1|
|    172457.25|    1|
|    167329.45|    1|
|    166143.17|    1|
|    165818.42|    1|
|    161653.36|    1|
|     161646.2|    1|
|    148411.45|    1|
|    148342.26|    1|
|    138953.37|    1|
|    138384.62|    1|
|    136619.47|    1|
|    130587.13|    1|
|    128870.92|    1|
|    127377.15|    1|
|    126501.77|    1|
|     125405.4|    1|
|     120206.7|    1|
|    118274.34|    1|
|    11717

These all looks like unrealistic trips, lets check the percentile values

In [72]:
print('The 90th percentile of trip distance is: ',df_new_1.approxQuantile('trip_distance',[0.90],relativeError=0.05)[0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The 90th percentile of trip distance is:  5.91

In [73]:
print('The 94th percentile of trip distance is: ',df_new_1.approxQuantile('trip_distance',[0.94],relativeError=0.05)[0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The 94th percentile of trip distance is:  8.76

In [74]:
df_new_1.filter(df_new_1['trip_distance']>8.76).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1644659

Around 6 % from the trips is above 8.76 miles . but still 8.76 mile per trip is realistic. we will just try to remove the unrealistic trips by removing `any trip above 1000 mile`

In [75]:
# Number of trips above or equal 1000 miles
df_new_1.filter(df_new_1['trip_distance']>=1000).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

190

In [76]:
df_new_1=df_new_1.filter(df_new_1['trip_distance']<1000)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
df_new_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24473993

## 2.3 Fare amount
As per https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page the min fare amount is $2.50 initial charge. so we will analyze the fare amount

In [78]:
# count of trips that have fare amount less than 2.5 dollars 
df_new_1.filter(df_new_1['fare_amount']<2.5).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

104476

In [79]:
#removing the trips that have fare amount less than 2.5
df_new_1=df_new_1.filter(df_new_1['fare_amount']>=2.5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
df_new_1.select('fare_amount').describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|       fare_amount|
+-------+------------------+
|  count|          24369517|
|   mean|12.669900323806095|
| stddev|263.56079833086795|
|    min|               2.5|
|    max|         998310.03|
+-------+------------------+

As we can see, the average fare amount is 12.6 $ but we have unrealistic fare_amount which need to be removed.

In [81]:
df_new_1.groupBy('fare_amount').count().orderBy(df_new_1['fare_amount'].desc()).show(50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+
|fare_amount|count|
+-----------+-----+
|  998310.03|    1|
|  671100.14|    1|
|  429496.72|    1|
|  187438.96|    1|
|  151504.45|    1|
|     6964.0|    1|
|     6052.0|    1|
|     4265.0|    1|
|     3014.5|    1|
|     1718.5|    1|
|     1333.0|    1|
|     1259.0|    1|
|     1238.0|    1|
|     1091.0|    1|
|     999.08|    1|
|      960.5|    1|
|      950.0|    3|
|      941.5|    1|
|      900.0|    1|
|      850.0|    1|
|      820.5|    1|
|      819.5|    1|
|      804.0|    1|
|      801.5|    1|
|      800.0|    4|
|      780.0|    1|
|      771.0|    1|
|      765.5|    1|
|      750.0|    3|
|      740.0|    1|
|      735.5|    1|
|      730.0|    1|
|      708.5|    1|
|      703.5|    1|
|      700.0|    9|
|      699.5|    1|
|      656.5|    1|
|     655.35|    4|
|      652.5|    1|
|      651.5|    1|
|      650.0|    3|
|      641.5|    1|
|     639.23|    1|
|      632.5|    1|
|      631.0|    1|
|      616.0|    1|
|      615.0|    1|


__Checking the outliers__

In [82]:
 # Q1 : First Quartile ., Q3 : Third Quartile
Q1 = df_new_1.approxQuantile('fare_amount',[0.25],relativeError=0.05)
Q3 = df_new_1.approxQuantile('fare_amount',[0.75],relativeError=0.05)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
#IQR : Inter Quantile Range
#We need to define the index [0], as Q1 & Q3 are a set of lists
IQR = Q3[0] - Q1[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
#selecting the data, with -1.5*IQR to + 1.5*IQR., where param = 1.5 default value
less_Q1 =  Q1[0] - 1.5*IQR
more_Q3 =  Q3[0] + 1.5*IQR

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
more_Q3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22.75

In [86]:
df_new_1.filter(df_new_1['fare_amount']>more_Q3).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2661686

We have 9.6% of our trips have fare amount 22.75 dollars and considered as outliers. but still we will keep them and remove the unrealistic values

In [87]:
df_new_1.filter(df_new_1['fare_amount']>500).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

__we will remove the trips `above 500 dollars` since its not realistic in my point of view and only 113 trips which will not affect our data__

In [88]:
df_new_1=df_new_1.filter(df_new_1['fare_amount']<=500)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
df_new_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24369404

## 2.4 Passenger

The number of passengers in the vehicle.
This is a driver-entered value.

In [90]:
df_new_1.select('passenger_count').describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|   passenger_count|
+-------+------------------+
|  count|          23572450|
|   mean|1.4709482467880937|
| stddev|1.1158997023789476|
|    min|               0.0|
|    max|               9.0|
+-------+------------------+

In [91]:
df_new_1.filter(df_new_1['passenger_count']==0).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

487096

Since its the taxi driver entry, maybe its was written by mistake. so will not remove the trips with zero passenger count

## 2.5 Trip duration
As per NYC Taxi & Limousine Commision Regulations, the maximum duration taxi driver can drive per day is 12 hours, so we will assume that the max trip duration could be 12 hours and remove and records axceed 12 hours

Steps to remove the trips above 12 hours:

* We will create new column that contain the trip duration by second `trip_duration_seconds` (just subtract the UNIX pickup time from the UNIX drop time)

* We will create trip_duration in hours from the newly created `trip_duration_seconds`

* Then filterout and remove any trip above 12 hours

In [92]:
# Create new coloumn trip_duration_seconds
df_new_1.withColumn('trip_duration_seconds',unix_timestamp("tpep_dropoff_datetime") - unix_timestamp('tpep_pickup_datetime'))\
.select(['tpep_pickup_datetime','tpep_dropoff_datetime','trip_duration_seconds']).show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+---------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|trip_duration_seconds|
+--------------------+---------------------+---------------------+
| 2020-04-01 00:41:22|  2020-04-01 01:01:53|                 1231|
| 2020-04-01 00:56:00|  2020-04-01 01:09:25|                  805|
| 2020-04-01 00:00:26|  2020-04-01 00:09:25|                  539|
| 2020-04-01 00:24:38|  2020-04-01 00:34:38|                  600|
| 2020-04-01 00:13:24|  2020-04-01 00:18:26|                  302|
| 2020-04-01 00:24:36|  2020-04-01 00:33:09|                  513|
| 2020-04-01 00:56:56|  2020-04-01 01:09:13|                  737|
| 2020-04-01 00:06:56|  2020-04-01 00:14:15|                  439|
| 2020-04-01 00:50:05|  2020-04-01 01:08:54|                 1129|
| 2020-04-01 00:07:10|  2020-04-01 00:18:45|                  695|
| 2020-04-01 00:37:21|  2020-04-01 00:46:00|                  519|
| 2020-04-01 00:19:44|  2020-04-01 00:22:27|                  

In [93]:
df_new_1=df_new_1.withColumn('trip_duration_seconds',unix_timestamp("tpep_dropoff_datetime") - unix_timestamp('tpep_pickup_datetime'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
#We will create trip duration in hour which is the `trip_duration_seconds / 3600`
df_new_1=df_new_1.withColumn('duration_In_Hours',round(col('trip_duration_seconds')/3600,1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
df_new_1.filter(df_new_1['duration_In_Hours']>12).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

45406

In [96]:
df_new_1=df_new_1.filter(df_new_1['duration_In_Hours']<=12)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
df_new_1.filter(df_new_1['duration_In_Hours']>12).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

## 2.6 Check the trips in each month

The trips in each month shpuld be dropped in the same month and some trips in the next month. for example january trips will have pick-up trips occurred in january and the drop of these trips must be in january and some of them will be dropped in february, following the assumption that some trips have been picked during the last hour in 31-Jan and it might been dropped in the first hour during the february first.

In [99]:
for i in range(1,13):
    print('Pick up month ',i,df_new_1.filter(df_new_1['Pick_month']==i).select('drop_month').distinct().collect())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pick up month  1 [Row(drop_month=1), Row(drop_month=2)]
Pick up month  2 [Row(drop_month=3), Row(drop_month=2)]
Pick up month  3 [Row(drop_month=3), Row(drop_month=4), Row(drop_month=2)]
Pick up month  4 [Row(drop_month=4), Row(drop_month=5)]
Pick up month  5 [Row(drop_month=6), Row(drop_month=5)]
Pick up month  6 [Row(drop_month=6), Row(drop_month=7)]
Pick up month  7 [Row(drop_month=8), Row(drop_month=7)]
Pick up month  8 [Row(drop_month=8), Row(drop_month=9)]
Pick up month  9 [Row(drop_month=9), Row(drop_month=10)]
Pick up month  10 [Row(drop_month=11), Row(drop_month=10)]
Pick up month  11 [Row(drop_month=12), Row(drop_month=11)]
Pick up month  12 [Row(drop_month=12), Row(drop_month=1)]

__We notice that in March trips we have trips dropped in feb which is incorrect. lets view this data__

In [100]:
df_new_1.filter((df_new_1['Pick_month']==3) & (df_new_1['drop_month']==2)).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [101]:
df_new_1.filter((df_new_1['Pick_month']==3) & (df_new_1['drop_month']==2)).select(['tpep_pickup_datetime','tpep_dropoff_datetime']).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------------------+---------------------+
| 2020-03-13 06:20:37|  2020-02-09 07:18:00|
+--------------------+---------------------+

its wrong trip which should be removed

__Steps to remove this trip:__

* create dataset without march

* create dataframe only for March trips

* filter out this row and union the two dataframe

In [103]:
df_new_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24323998

In [104]:
df_new_1.filter(df_new_1['Pick_month']==3).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2976075

In [105]:
df_new_1.filter(df_new_1['Pick_month']!=3).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

21347923

In [106]:
# create march_df
march=df_new_1.filter(df_new_1['Pick_month']==3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
# create no_march_df
no_march_df = df_new_1.filter(df_new_1['Pick_month']!=3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
# filter out the feb drop row
march.filter(march['drop_month']==2).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [109]:
march=march.filter(march['drop_month']!=2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
march.select('drop_month').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(drop_month=3), Row(drop_month=4)]

In [111]:
df_new_final = march.union(no_march_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
df_new_final.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24323997

In [113]:
df_new_1.count() - df_new_final.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [114]:
# check the drop year for the trips happened in Dec 2020
df_new_final.filter(df_new_final['Pick_month']==12).select('Drop_Year').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Drop_Year=2020), Row(Drop_Year=2021)]

__Everything look cleaned now__

we will move to group and aggregate the trips data

# 3- Grouping the data per location ID in Hourly basis and Make it ready for Modeling

__We will need to groupby and aggregate the count of the taxi trips per location ID  to be in hourly basis__

Lets have look how the data look like

In [115]:
df_new_final.select(['tpep_pickup_datetime','PULocationID']).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|tpep_pickup_datetime|PULocationID|
+--------------------+------------+
| 2020-03-16 06:17:14|          24|
| 2020-03-04 10:12:32|          43|
| 2020-03-25 18:51:29|          43|
| 2020-03-25 19:24:19|         230|
| 2020-03-26 04:02:21|         137|
| 2020-03-07 12:35:20|          79|
| 2020-03-07 13:06:13|         107|
| 2020-03-07 13:40:17|         264|
| 2020-03-07 13:41:34|         236|
| 2020-03-07 14:11:24|         230|
| 2020-03-07 14:29:23|         239|
| 2020-03-07 14:46:16|         230|
| 2020-03-07 15:49:30|         170|
| 2020-03-07 16:53:56|         261|
| 2020-03-08 06:02:16|         114|
| 2020-03-08 06:23:47|         142|
| 2020-03-08 07:00:15|         186|
| 2020-03-08 08:05:36|         170|
| 2020-03-08 09:21:57|         148|
| 2020-03-08 10:10:08|          13|
+--------------------+------------+
only showing top 20 rows

__Lets check how many Location ID available in our dataframe__

In [116]:
df_new_final.select(countDistinct("PULocationID")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+
|count(DISTINCT PULocationID)|
+----------------------------+
|                         262|
+----------------------------+

__We have 262 location ID, so when we will aggregate the the taxi trips in hourly basis, we should have `366 X 24 X 262 = 2301408 rows`__

- 366 days in 2020

- 24 hours per day

- 262 location ID

### Steps:
We will change the timestamp to this format `"yyyy-MM-dd HH:00"` to be able to group the trips hourly

In [117]:
df_new_final=df_new_final.withColumn("Pickup_datetime_hourly", date_format(col("tpep_pickup_datetime").cast("timestamp"), "yyyy-MM-dd HH:00"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
df_new_final.select(['tpep_pickup_datetime','Pickup_datetime_hourly','PULocationID']).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------------+------------+
|tpep_pickup_datetime|Pickup_datetime_hourly|PULocationID|
+--------------------+----------------------+------------+
| 2020-03-16 06:17:14|      2020-03-16 06:00|          24|
| 2020-03-04 10:12:32|      2020-03-04 10:00|          43|
| 2020-03-25 18:51:29|      2020-03-25 18:00|          43|
| 2020-03-25 19:24:19|      2020-03-25 19:00|         230|
| 2020-03-26 04:02:21|      2020-03-26 04:00|         137|
| 2020-03-07 12:35:20|      2020-03-07 12:00|          79|
| 2020-03-07 13:06:13|      2020-03-07 13:00|         107|
| 2020-03-07 13:40:17|      2020-03-07 13:00|         264|
| 2020-03-07 13:41:34|      2020-03-07 13:00|         236|
| 2020-03-07 14:11:24|      2020-03-07 14:00|         230|
| 2020-03-07 14:29:23|      2020-03-07 14:00|         239|
| 2020-03-07 14:46:16|      2020-03-07 14:00|         230|
| 2020-03-07 15:49:30|      2020-03-07 15:00|         170|
| 2020-03-07 16:53:56|      2020-03-07 16:00|         26

__we will create trip count column__

In [118]:
# we will create trip count column
df_new_final=df_new_final.withColumn("Trip_count", lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Now lets groupby and aggregate the trips per location in hourly basis

In [119]:
hourly_aggregated=df_new_final.groupby(['Pickup_datetime_hourly','PULocationID']).agg({'Trip_count':'count'})\
.withColumnRenamed("count(Trip_count)", "Trips_count")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
hourly_aggregated.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+------------+-----------+
|Pickup_datetime_hourly|PULocationID|Trips_count|
+----------------------+------------+-----------+
|      2020-04-01 19:00|         186|          4|
|      2020-04-02 13:00|          33|          1|
|      2020-04-03 07:00|          79|         17|
|      2020-04-03 07:00|         233|          3|
|      2020-04-03 14:00|          74|         14|
|      2020-04-06 15:00|         264|          4|
|      2020-04-07 00:00|         256|          1|
|      2020-04-08 21:00|          87|          2|
|      2020-04-09 10:00|         237|         20|
|      2020-04-10 19:00|         164|          3|
|      2020-04-12 12:00|         236|         12|
|      2020-04-12 12:00|          79|          5|
|      2020-04-12 17:00|         181|          1|
|      2020-04-12 22:00|         229|          2|
|      2020-04-13 13:00|          74|         10|
|      2020-04-13 17:00|         213|          1|
|      2020-04-14 07:00|         166|          5|


In [121]:
hourly_aggregated.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

910784

__The aggregated trips have `910784` rows and it should be `2301408`, and I think the issue is there are no pick-up records in certain hours. So when I gourpby and aggregate the data, it will not show the missing hours. 

#### To solve this problem:
- we need to create dataframe with timestamp coloumn started from 1-1-2020 till 1-1-2021. `(8784 rows)`.
- We will use the timestamp dataframe which we create in the first step and use it to generate dataframe that contains timestamp column and another coloumn per location ID and then merge all of them. in this case we will have `(2301408)` rows and two columns, timestamp and LocationID.
- Create unique column that contain the __timestamp plus the locationID__ in both `hourly_aggregated` and ` generated timestamp dataframe` (the dataframe we created in the previous step) so we can join them.
- Then left join the generated timestamp dataframe with our hourly aggregated dataframe.

### 1- Creating unique coloum in hourly_aggregated
The unique column will merge the timestamp with the locationID. we need to convert the locationID from integer to string then we will use `concat_ws` to merge the string values together

In [127]:
hourly_aggregated.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Pickup_datetime_hourly: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- Trips_count: long (nullable = false)

In [128]:
from pyspark.sql.types import TimestampType,StructField,StringType,IntegerType,StructType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
#Create new column which convert the location ID to string
hourly_aggregated= hourly_aggregated.withColumn("PULocationID_string", hourly_aggregated["PULocationID"].cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
#Create the unique column using concat_ws
hourly_aggregated=hourly_aggregated.select(concat_ws('_',hourly_aggregated.Pickup_datetime_hourly,
                                   hourly_aggregated.PULocationID_string).alias("UniqueColumn"),"Pickup_datetime_hourly","PULocationID","Trips_count")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
hourly_aggregated.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------------+------------+-----------+
|        UniqueColumn|Pickup_datetime_hourly|PULocationID|Trips_count|
+--------------------+----------------------+------------+-----------+
|2020-04-01 19:00_186|      2020-04-01 19:00|         186|          4|
| 2020-04-02 13:00_33|      2020-04-02 13:00|          33|          1|
| 2020-04-03 07:00_79|      2020-04-03 07:00|          79|         17|
|2020-04-03 07:00_233|      2020-04-03 07:00|         233|          3|
| 2020-04-03 14:00_74|      2020-04-03 14:00|          74|         14|
+--------------------+----------------------+------------+-----------+
only showing top 5 rows

In [132]:
hourly_aggregated.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- UniqueColumn: string (nullable = false)
 |-- Pickup_datetime_hourly: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- Trips_count: long (nullable = false)

### 2- Generate timestamp dataframe from 1-1-2020 till 1-1-2021
The below function will generate spark timestamp dataframe 

In [133]:
def generate_series(start, stop, interval):
    """
    :param start  - lower bound, inclusive
    :param stop   - upper bound, exclusive
    :interval int - increment interval in seconds
    """
    start, stop = spark.createDataFrame(
        [(start, stop)], ("start", "stop")
    ).select(
        [col(c).cast("timestamp").cast("long") for c in ("start", "stop")
    ]).first()
    # Create range with increments and cast to timestamp
    return spark.range(start, stop, interval).select(
        col("id").cast("timestamp").alias("timestamp")
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
# we will multiply by 60*60 to change the timestamp to hour
timestamp_df=generate_series("2020-01-01", "2021-01-01", 60 * 60)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__We will change the generated timestamp to the same format in `hourly_aggregated`__

In [135]:
timestamp_df=timestamp_df.withColumn("hourly_timestamp", date_format(col("timestamp").cast("timestamp"), "yyyy-MM-dd HH:00"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
timestamp_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- timestamp: timestamp (nullable = false)
 |-- hourly_timestamp: string (nullable = false)

In [137]:
timestamp_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8784

__Now this dataframe contain hourly timestamp for 2020. so we need to create the same rows for each location ID exist in `hourly_aggregated`__

Lets extract the location ID from the hourly_aggregated and save them into list

In [138]:
# extract the pickup location id from the dataset and save it in list
Pick_up_LocationID = df_new_final.select(collect_set('PULocationID').alias('locationID_list')).first()['locationID_list']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
len(Pick_up_LocationID)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

262

__We have 262 location ID, we will aggregate the trips in hourly basis. so we should have `366 X 24 X 262 = 2301408` rows.__

To do so, we need to iterate generated timestamp for each locationID (for loop) and append the result to an empty spark dataframe. in this case, we will have 2 columns and 2301408 rows.

__Define function that will help us to add the location ID column to the generated timestamp__

In [141]:
def locations_timestampGenerator(dataframe,locationID):
    dataframe=dataframe.withColumn("locationID", lit(locationID))
    return dataframe

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Now lets create an empty spark dataframe contains two columns `timestamp` and `locationID`__

In [142]:
data_schema = [StructField("timestamp",TimestampType(), True),StructField("hourly_timestamp", StringType(), True),
               StructField("locationID", IntegerType(), True)]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
final_struct = StructType(fields=data_schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
ID_plus_timestamp = spark.createDataFrame([], schema=final_struct)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
# iterate generated timestamp for each locationID (for loop) 
for i in Pick_up_LocationID:
    dataframe=locations_timestampGenerator(timestamp_df,i)
    ID_plus_timestamp = ID_plus_timestamp.union(dataframe)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
ID_plus_timestamp.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2301408

__Now lets make sure that all the locationID created by the `for loop` is matching the existing locationID in the original aggregated dataset__

We will extract the distinct location ID from `ID_plus_timestamp` and check with the location ID in `Hourly_aggregated`

In [147]:
# extract the distinct location ID from ID_plus_timestamp
location_ID_list2=ID_plus_timestamp.select(collect_set('locationID').alias('locationID_list')).first()['locationID_list']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Remember we already extracted the location ID from Hourly_aggregated `Pick_up_LocationID`, so we dont need to do it again__

now if there is any location ID in `ID_plus_timestamp`dataframe not exist in `Hourly_aggregated` it will append it to the empty list

In [148]:
unique_list = []
 
for item in location_ID_list2: 
    if item not in Pick_up_LocationID: 
        unique_list.append(item)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
# Should be empty
unique_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

#### Last step, We will need to create unique column that contain the __timestamp plus the locationID__ in `ID_plus_timestamp` so we can join it to `hourly_aggregated`.

In [150]:
# Change the location ID from integer to string
ID_plus_timestamp= ID_plus_timestamp.withColumn("locationID_string", ID_plus_timestamp["locationID"].cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
# Create the unique column using concat_ws
ID_plus_timestamp=ID_plus_timestamp.select(concat_ws('_',ID_plus_timestamp.hourly_timestamp,
                                   ID_plus_timestamp.locationID_string).alias("UniqueColumn_G"),"timestamp","hourly_timestamp","locationID")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
ID_plus_timestamp.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+----------------+----------+
|      UniqueColumn_G|          timestamp|hourly_timestamp|locationID|
+--------------------+-------------------+----------------+----------+
|2020-01-01 00:00_256|2020-01-01 00:00:00|2020-01-01 00:00|       256|
|2020-01-01 01:00_256|2020-01-01 01:00:00|2020-01-01 01:00|       256|
|2020-01-01 02:00_256|2020-01-01 02:00:00|2020-01-01 02:00|       256|
+--------------------+-------------------+----------------+----------+
only showing top 3 rows

#### Now joining both dataframes using the `unique column`

In [153]:
# join both dataframes using the unique column
Pick_aggregated=hourly_aggregated.join(ID_plus_timestamp,hourly_aggregated.UniqueColumn ==  ID_plus_timestamp.UniqueColumn_G,"right")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Now after we joined the dataframes, we will need to replace null values in PULocationID and count(Trip_count) columns of dataframe hourly_aggregated_final with 0.__

Assume  PULocationID of missing data to be 0

In [154]:
Pick_aggregated=Pick_aggregated.na.fill(0,subset=["PULocationID"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
Pick_aggregated=Pick_aggregated.na.fill(0,subset=["Trips_count"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Now lets select the columns we need : `timestamp`, `locationID` ,` Trips_count` and the `UniqueColumn_G`
I have selected `UniqueColumn_G` to be the unique identifier and to join it with the dropped aggregated dataframe

In [156]:
Pick_aggregated=Pick_aggregated.select(['timestamp','hourly_timestamp','locationID','Trips_count','UniqueColumn_G'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
Pick_aggregated.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+----------+-----------+--------------------+
|          timestamp|hourly_timestamp|locationID|Trips_count|      UniqueColumn_G|
+-------------------+----------------+----------+-----------+--------------------+
|2020-01-01 00:00:00|2020-01-01 00:00|       150|          0|2020-01-01 00:00_150|
|2020-01-01 00:00:00|2020-01-01 00:00|       189|         10|2020-01-01 00:00_189|
|2020-01-01 00:00:00|2020-01-01 00:00|       195|          0|2020-01-01 00:00_195|
|2020-01-01 00:00:00|2020-01-01 00:00|       233|        118|2020-01-01 00:00_233|
|2020-01-01 00:00:00|2020-01-01 00:00|       241|          1|2020-01-01 00:00_241|
|2020-01-01 00:00:00|2020-01-01 00:00|        95|          1| 2020-01-01 00:00_95|
|2020-01-01 01:00:00|2020-01-01 01:00|       119|          0|2020-01-01 01:00_119|
|2020-01-01 01:00:00|2020-01-01 01:00|       124|          0|2020-01-01 01:00_124|
|2020-01-01 01:00:00|2020-01-01 01:00|        13|         65| 2020-01-01 01:00_13|
|202

## Aggregating the drop trips per location ID in hourly basis

- We will follow the same steps how we aggregate the pick-up trips above, but now for the drop-off so we will have hourly count per location ID for the pick-up trips and drop-off trips

In [158]:
df_new_final.select(countDistinct("DOLocationID")).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+
|count(DISTINCT DOLocationID)|
+----------------------------+
|                         263|
+----------------------------+

__We have 263 drop location ID, and we have 262 pick-up location ID, we will remove the drop trips which belong to the extra location ID__

In [159]:
# extract the pickup location id from the dataset and save it in list
Drop_LocationID = df_new_final.select(collect_set('DOLocationID').alias('DOLocationID')).first()['DOLocationID']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
unique_list = []
 
for item in Drop_LocationID: 
    if item not in Pick_up_LocationID: 
        unique_list.append(item)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
unique_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[104]

so we will remove the drop trips for location ID = 104

In [162]:
Drop_df=df_new_final.filter(df_new_final['DOLocationID']!=104)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Now we are going to aggregate the data based on hourly drop count per locationID.
We will change the timestamp to this format `"yyyy-MM-dd HH:00"` to be able to group the trips hourly

In [163]:
Drop_df=Drop_df.withColumn("Drop_datetime_hourly", date_format(col("tpep_dropoff_datetime").cast("timestamp"), "yyyy-MM-dd HH:00"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
# Grouping and aggregating the dropped trips
drop_aggregated = Drop_df.groupby(['Drop_datetime_hourly','DOLocationID']).agg({'Trip_count':'count'})\
.withColumnRenamed("count(Trip_count)", "Drop_Trips_count")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [165]:
drop_aggregated.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+----------------+
|Drop_datetime_hourly|DOLocationID|Drop_Trips_count|
+--------------------+------------+----------------+
|    2020-04-01 19:00|         186|               1|
|    2020-04-03 06:00|         125|               1|
|    2020-04-03 07:00|         233|               9|
|    2020-04-03 07:00|          79|               4|
|    2020-04-03 13:00|          66|               3|
|    2020-04-03 14:00|          74|              23|
|    2020-04-03 16:00|         220|               1|
|    2020-04-04 02:00|          72|               1|
|    2020-04-06 15:00|         264|               3|
|    2020-04-06 23:00|          47|               1|
|    2020-04-07 00:00|         256|               1|
|    2020-04-07 20:00|         226|               2|
|    2020-04-08 15:00|         127|               1|
|    2020-04-09 10:00|         237|              23|
|    2020-04-09 22:00|         125|               1|
|    2020-04-10 05:00|         164|           

__Create unique column that contain the timestamp plus the locationID in ID_plus_timestamp so we can join it to Pick_aggregated__

Convert the DOLocationID to string type

In [169]:
drop_aggregated=drop_aggregated.withColumn("DOLocationID_string", drop_aggregated["DOLocationID"].cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
# Create the unique column using `concat_ws`
drop_aggregated=drop_aggregated.select(concat_ws('_',drop_aggregated.Drop_datetime_hourly,
                                   drop_aggregated.DOLocationID_string).alias("UniqueColumn"),"Drop_datetime_hourly","Drop_datetime_hourly","DOLocationID","Drop_Trips_count")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Joining Pick_aggregated & drop_aggregated dataframes using the UniqueColumn

In [171]:
final_df=drop_aggregated.join(Pick_aggregated,drop_aggregated.UniqueColumn == Pick_aggregated.UniqueColumn_G,"right")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [172]:
# Filling the null Drop_Trips_count with zero
final_df=final_df.na.fill(0,subset=["Drop_Trips_count"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Now we will just select the columns we  want:

`hourly timestamp`, `locationID`, `Pick_Trips_count` and `Drop_Trips_count`

In [173]:
final_df=final_df.select(['timestamp','hourly_timestamp','locationID','Trips_count','Drop_Trips_count'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
final_df=final_df.withColumnRenamed('Trips_count','Pick_Trips_count')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [175]:
final_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+----------+----------------+----------------+
|          timestamp|hourly_timestamp|locationID|Pick_Trips_count|Drop_Trips_count|
+-------------------+----------------+----------+----------------+----------------+
|2020-01-01 00:00:00|2020-01-01 00:00|       150|               0|               0|
|2020-01-01 00:00:00|2020-01-01 00:00|       189|              10|              16|
|2020-01-01 00:00:00|2020-01-01 00:00|       195|               0|               2|
|2020-01-01 00:00:00|2020-01-01 00:00|       233|             118|             173|
|2020-01-01 00:00:00|2020-01-01 00:00|       241|               1|               2|
|2020-01-01 00:00:00|2020-01-01 00:00|        95|               1|              16|
|2020-01-01 01:00:00|2020-01-01 01:00|       119|               0|              11|
|2020-01-01 01:00:00|2020-01-01 01:00|       124|               0|               1|
|2020-01-01 01:00:00|2020-01-01 01:00|        13|              65|          

# Feature Engineering

Now the dataframe is ready for modeling, final step is just to create some features such as month, day, year, hour and dayofweek


In [176]:
# Create Year column
final_df=final_df.withColumn('Year',year(final_df['timestamp']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
# Create Month column
final_df=final_df.withColumn('Month',month(final_df['timestamp']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
# create day of month column
final_df=final_df.withColumn('DayOfMonth',dayofmonth(final_df['timestamp']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
# Create hour column
final_df=final_df.withColumn('Hour',hour(final_df['timestamp']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [180]:
# Create Day of week column
final_df=final_df.withColumn("dayofweek", dayofweek(col("timestamp")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
final_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+----------+----------------+----------------+----+-----+----------+----+---------+
|          timestamp|hourly_timestamp|locationID|Pick_Trips_count|Drop_Trips_count|Year|Month|DayOfMonth|Hour|dayofweek|
+-------------------+----------------+----------+----------------+----------------+----+-----+----------+----+---------+
|2020-01-01 00:00:00|2020-01-01 00:00|       150|               0|               0|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       189|              10|              16|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       195|               0|               2|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       233|             118|             173|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       241|               1|               2|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 

### Save the final dataframe as parquet file in S3

__Now we will save this spark dataframe to S3 bucket inside `EMR-project folder`__

In [182]:
final_df.coalesce(1).write.parquet("s3://taxi-project-thesis/EMR-project/final_2020")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
# CHeck if we can load the final_2020 dataframe 

In [184]:
### Load the final dataframe from S3
df_taxi=spark.read.parquet("s3://taxi-project-thesis/EMR-project/final_2020/part-00000-6b94e604-fe74-416d-a81c-d4efd46dd9cb-c000.snappy.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [185]:
df_taxi.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+----------+----------------+----------------+----+-----+----------+----+---------+
|          timestamp|hourly_timestamp|locationID|Pick_Trips_count|Drop_Trips_count|Year|Month|DayOfMonth|Hour|dayofweek|
+-------------------+----------------+----------+----------------+----------------+----+-----+----------+----+---------+
|2020-01-01 00:00:00|2020-01-01 00:00|       150|               0|               0|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       189|              10|              16|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       195|               0|               2|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       233|             118|             173|2020|    1|         1|   0|        4|
|2020-01-01 00:00:00|2020-01-01 00:00|       241|               1|               2|2020|    1|         1|   0|        4|
+-------------------+-----------

In [186]:
df_taxi.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2301408

DONE !